<a href="https://colab.research.google.com/github/WibuSOS/live-chat-translator/blob/ai-branch/production_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import unicodedata
import re
import math
from io import open
import numpy as np
import matplotlib.pyplot as plt
import pickle

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
# import torch.cuda

# this line clears sys to allow for argparse to work as gradient clipper
import sys; sys.argv=['']; del sys

In [ ]:
# This function converts a Unicode string to plain ASCII 
# from https://stackoverflow.com/a/518232/2809427
# def uniToAscii(sentence):
#     return ''.join(
#         c for c in unicodedata.normalize('NFD', sentence)
#         if unicodedata.category(c) != 'Mn'
#     )

# Lowercase, trim, and remove non-letter characters (from pytorch)
def normalizeString(s):
    s = re.sub(r" ##AT##-##AT## ", r" ", s)
    s = s.lower().strip()
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [ ]:
# start of sentence tag
SOS_token = 0

# end of sentence tag
EOS_token = 1

# unknown word tag (this is used to handle words that are not in our Vocabulary)
UNK_token = 2

# Lang class, used to store the vocabulary of each language
class Lang:
    def __init__(self, language):
        self.language_name = language
        self.word_to_index = {"SOS":SOS_token, "EOS":EOS_token, "<UNK>":UNK_token}
        self.word_to_count = {}
        self.index_to_word = {SOS_token: "SOS", EOS_token: "EOS", UNK_token: "<UNK>"}
        self.vocab_size = 3
        self.cutoff_point = -1

    def countSentence(self, sentence):
        for word in sentence.split(' '):
            self.countWords(word)

    # counts the number of times each word appears in the dataset
    def countWords(self, word):
        if word not in self.word_to_count:
            self.word_to_count[word] = 1
        else:
            self.word_to_count[word] += 1

    # if the number of unique words in the dataset is larger than the
    # specified max_vocab_size, creates a cutoff point that is used to
    # leave infrequent words out of the vocabulary
    def createCutoff(self, max_vocab_size):
        word_freqs = list(self.word_to_count.values())
        word_freqs.sort(reverse=True)
        if len(word_freqs) > max_vocab_size:
            self.cutoff_point = word_freqs[max_vocab_size]

    # assigns each unique word in a sentence a unique index
    def addSentence(self, sentence):
        new_sentence = ''
        for word in sentence.split(' '):
            unk_word = self.addWord(word)
            if not new_sentence:
                new_sentence = unk_word
            else:
                new_sentence = new_sentence + ' ' + unk_word
        return new_sentence

    # assigns a word a unique index if not already in vocabulary
    # and it appeaars often enough in the dataset
    # (self.word_to_count is larger than self.cutoff_point)
    def addWord(self, word):
        if self.word_to_count[word] > self.cutoff_point:
            if word not in self.word_to_index:
                self.word_to_index[word] = self.vocab_size
                self.index_to_word[self.vocab_size] = word
                self.vocab_size += 1
            return word
        else:
            return self.index_to_word[2]

In [ ]:
# converts a sentence to one hot encoding vectors - pytorch allows us to just
# use the number corresponding to the unique index for that word,
# rather than a complete one hot encoding vector for each word

def indexesFromSentence(lang, sentence):
    indexes = []
    for word in sentence.split(' '):
        try:
            indexes.append(lang.word_to_index[word])
        except:
            indexes.append(lang.word_to_index["<UNK>"])
    return indexes

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    result = torch.LongTensor(indexes).view(-1)
    if use_cuda:
        return result.cuda()
    else:
        return result

# converts from tensor of one hot encoding vector indices to sentence
# def sentenceFromTensor(lang, tensor):
#     raw = tensor.data
#     words = []
#     for num in raw:
#         words.append(lang.index_to_word[num.item()])
#     return ' '.join(words)

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self,input_size,hidden_size,layers=1,dropout=0.1,
                bidirectional=True):
        super(EncoderRNN, self).__init__()

        if bidirectional:
            self.directions = 2
        else:
            self.directions = 1
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = layers
        self.dropout = dropout
        self.embedder = nn.Embedding(input_size,hidden_size)
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(input_size=hidden_size,hidden_size=hidden_size,
                        num_layers=layers,dropout=dropout,
                        bidirectional=bidirectional,batch_first=False)
        self.fc = nn.Linear(hidden_size*self.directions, hidden_size)

    def forward(self, input_data, h_hidden, c_hidden):
        embedded_data = self.embedder(input_data)
        embedded_data = self.dropout(embedded_data)
        hiddens, outputs = self.lstm(embedded_data, (h_hidden, c_hidden))

        return hiddens, outputs

    # creates initial hidden states for encoder corresponding to batch size
    def create_init_hiddens(self, batch_size):
        h_hidden = Variable(torch.zeros(self.num_layers*self.directions, 
                                    batch_size, self.hidden_size))
        c_hidden = Variable(torch.zeros(self.num_layers*self.directions, 
                                    batch_size, self.hidden_size))
        
        return h_hidden, c_hidden

In [ ]:
class DecoderAttn(nn.Module):
	def __init__(self, hidden_size, output_size, layers=1, dropout=0.1, bidirectional=True):
		super(DecoderAttn, self).__init__()

		if bidirectional:
			self.directions = 2
		else:
			self.directions = 1
		self.output_size = output_size
		self.hidden_size = hidden_size
		self.num_layers = layers
		self.dropout = dropout
		self.embedder = nn.Embedding(output_size,hidden_size)
		self.dropout = nn.Dropout(dropout)
		self.score_learner = nn.Linear(hidden_size*self.directions, 
                                   hidden_size*self.directions)
		self.lstm = nn.LSTM(input_size=hidden_size,hidden_size=hidden_size,
                        num_layers=layers,dropout=dropout,
                        bidirectional=bidirectional,batch_first=False)
		self.context_combiner = nn.Linear((hidden_size*self.directions)
                                      +(hidden_size*self.directions), hidden_size)
		self.tanh = nn.Tanh()
		self.output = nn.Linear(hidden_size, output_size)
		self.soft = nn.Softmax(dim=1)
		self.log_soft = nn.LogSoftmax(dim=1)

	def forward(self, input_data, h_hidden, c_hidden, encoder_hiddens):
		embedded_data = self.embedder(input_data)
		embedded_data = self.dropout(embedded_data)	
		batch_size = embedded_data.shape[1]
		hiddens, outputs = self.lstm(embedded_data, (h_hidden, c_hidden))	
		top_hidden = outputs[0].view(self.num_layers,self.directions,
                                 hiddens.shape[1],
                                 self.hidden_size)[self.num_layers-1]
		top_hidden = top_hidden.permute(1,2,0).contiguous().view(batch_size,-1,1)

		prep_scores = self.score_learner(encoder_hiddens.permute(1,0,2))
		scores = torch.bmm(prep_scores, top_hidden)
		attn_scores = self.soft(scores)
		con_mat = torch.bmm(encoder_hiddens.permute(1,2,0),attn_scores)
		h_tilde = self.tanh(self.context_combiner(torch.cat((con_mat,
                                                         top_hidden),dim=1)
                                              .view(batch_size,-1)))
		pred = self.output(h_tilde)
		pred = self.log_soft(pred)

		return pred, outputs

In [ ]:
# Returns the predicted translation of a given input sentence. Predicted
# translation is trimmed to length of cutoff_length argument

def evaluate(encoder, decoder, input_lang, output_lang, sentence, cutoff_length):
    with torch.no_grad():
        input_variable = tensorFromSentence(input_lang, sentence)
        input_variable = input_variable.view(-1,1)

        enc_h_hidden, enc_c_hidden = encoder.create_init_hiddens(1)
        enc_hiddens, enc_outputs = encoder(input_variable, enc_h_hidden, enc_c_hidden)

        decoder_input = Variable(torch.LongTensor(1,1).fill_(output_lang.word_to_index.get("SOS")).cuda()) if use_cuda \
                        else Variable(torch.LongTensor(1,1).fill_(output_lang.word_to_index.get("SOS")))
        dec_h_hidden = enc_outputs[0]
        dec_c_hidden = enc_outputs[1]

        decoded_words = []

        for di in range(cutoff_length):
            pred, dec_outputs = decoder(decoder_input, dec_h_hidden, dec_c_hidden, enc_hiddens)
            topv, topi = pred.topk(1,dim=1)
            ni = topi.item()

            if ni == output_lang.word_to_index.get("EOS"):
                # decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index_to_word[ni])

            decoder_input = Variable(torch.LongTensor(1,1).fill_(ni).cuda()) if use_cuda \
                            else Variable(torch.LongTensor(1,1).fill_(ni))
            dec_h_hidden = dec_outputs[0]
            dec_c_hidden = dec_outputs[1]

        output_sentence = ' '.join(decoded_words)

        return output_sentence

In [ ]:
# HYPERPARAMETERS: FEEL FREE TO PLAY WITH THESE TO TRY TO ACHIEVE BETTER RESULTS

# signifies whether the Encoder and Decoder should be bidirectional LSTMs or not
bidirectional = True
if bidirectional:
	directions = 2
else:
	directions = 1

# number of layers in both the Encoder and Decoder
layers = 2

# Hidden size of the Encoder and Decoder
hidden_size = 600

# Dropout value for Encoder and Decoder
dropout = 0.6

In [ ]:
# LOAD CONFIGURATIONS

# Set the common name of the loading files
common_file_name = "testdata.tatoeba_identic_trim.20_vocab.25000_directions.2_layers.4_hidden.100_dropout.0.5_learningrate.1_batch.10_epochs.100"
id_lang = 'id'
en_lang = 'en'
dataset = 'tatoeba_identic'
id_en_directory = ''
en_id_directory = ''

# Set these configurations if you want to load models from google drive
load_from_drive = True
if load_from_drive:
    common_file_name = "testdata>tatoeba_identic_trim>20_vocab>25000_directions>2_layers>4_hidden>100_dropout>0.5_learningrate>1_batch>10_epochs>100"
    super_directory = '/content/drive/'
    id_en_experiment_directory = 'MyDrive/Kuliah/Thesis/experiment/id_en/'
    en_id_experiment_directory = 'MyDrive/Kuliah/Thesis/experiment/en_id/'
    id_en_directory = super_directory + id_en_experiment_directory
    en_id_directory = super_directory + en_id_experiment_directory

# Set the name of the loading files
id_vocab_file = id_en_directory + id_lang + '_21484_' + dataset + '_vocab.p'
en_vocab_file = id_en_directory + en_lang + '_21731_' + dataset + '_vocab.p'
id_en_enc_file = '%s%s_%s_enc_direction_%s_layer_%s_hidden_%s_dropout_%s.pth' % (id_en_directory, id_lang, en_lang, directions, layers, hidden_size, dropout)
id_en_dec_file = '%s%s_%s_dec_direction_%s_layer_%s_hidden_%s_dropout_%s.pth' % (id_en_directory, id_lang, en_lang, directions, layers, hidden_size, dropout)
en_id_enc_file = '%s%s_%s_enc_direction_%s_layer_%s_hidden_%s_dropout_%s.pth' % (en_id_directory, en_lang, id_lang, directions, layers, hidden_size, dropout)
en_id_dec_file = '%s%s_%s_dec_direction_%s_layer_%s_hidden_%s_dropout_%s.pth' % (en_id_directory, en_lang, id_lang, directions, layers, hidden_size, dropout)

# Mandatory variables initialization
device = torch.device('cpu')
# use_cuda = torch.cuda.is_available()
use_cuda = False
id_vocab = None
en_vocab = None
id_en_encoder = None
id_en_decoder = None
en_id_encoder = None
en_id_decoder = None

In [ ]:
# LOAD FROM DRIVE: OPTIONAL!!!
# execute this cell if you want to load models from google drive

if load_from_drive:
    from google.colab import drive
    drive.mount(super_directory)

Mounted at /content/drive/


In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
# LOAD EVERYTHING

id_vocab = pickle.load(open(id_vocab_file,'rb'))
en_vocab = pickle.load(open(en_vocab_file,'rb'))

id_en_encoder = EncoderRNN(id_vocab.vocab_size, hidden_size, layers=layers, 
                           dropout=dropout, bidirectional=bidirectional)
id_en_decoder = DecoderAttn(hidden_size, en_vocab.vocab_size, layers=layers, 
                            dropout=dropout, bidirectional=bidirectional)

en_id_encoder = EncoderRNN(en_vocab.vocab_size, hidden_size, layers=layers, 
                           dropout=dropout, bidirectional=bidirectional)
en_id_decoder = DecoderAttn(hidden_size, id_vocab.vocab_size, layers=layers, 
                            dropout=dropout, bidirectional=bidirectional)

id_en_encoder.load_state_dict(torch.load(id_en_enc_file, map_location=device))
id_en_decoder.load_state_dict(torch.load(id_en_dec_file, map_location=device))

en_id_encoder.load_state_dict(torch.load(en_id_enc_file, map_location=device))
en_id_decoder.load_state_dict(torch.load(en_id_dec_file, map_location=device))

id_en_encoder.eval()
id_en_decoder.eval()

en_id_encoder.eval()
en_id_decoder.eval()

DecoderAttn(
  (embedder): Embedding(21484, 600)
  (dropout): Dropout(p=0.6, inplace=False)
  (score_learner): Linear(in_features=1200, out_features=1200, bias=True)
  (lstm): LSTM(600, 600, num_layers=2, dropout=0.6, bidirectional=True)
  (context_combiner): Linear(in_features=2400, out_features=600, bias=True)
  (tanh): Tanh()
  (output): Linear(in_features=600, out_features=21484, bias=True)
  (soft): Softmax(dim=1)
  (log_soft): LogSoftmax(dim=1)
)

In [ ]:
# Test a sentence outside the dataset
running = True
while running:
    command = input()
    if command.lower() == 'e':
        running = False
    elif command.lower() == 'id_en':
        input_sentence = input()
        print(evaluate(id_en_encoder, id_en_decoder, id_vocab, en_vocab, normalizeString(input_sentence), cutoff_length=10))
    elif command.lower() == 'en_id':
        input_sentence = input()
        print(evaluate(en_id_encoder, en_id_decoder, en_vocab, id_vocab, normalizeString(input_sentence), cutoff_length=10))

id_en
gue

en_id
bro
.
